In [75]:
#importing:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
import numpy as np
import json

In [2]:
pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -U feature-engine

Requirement already up-to-date: feature-engine in c:\users\tl\anaconda3\lib\site-packages (1.1.2)
Note: you may need to restart the kernel to use updated packages.


In [222]:
#load file:
data = pd.read_json("train-1.json")
df=pd.DataFrame(data)
test = pd.read_json("test.json")
df_test=pd.DataFrame(test)
#df.head()
df_test.head()

,is_open_access,doi,title,fields_of_study,abstract,year,venue,references,authors,topics
0,False,10.18653/v1/2021.findings-acl.255,On Sparsifying Encoder Outputs in Sequence-to-...,[Computer Science],Sequence-to-sequence models usually transfer a...,2021,FINDINGS,47,"[Biao Zhang, Ivan Titov, Rico Sennrich]","[Encoder, Transformer, Automatic summarization..."
1,True,10.18653/v1/2020.acl-main.200,To Pretrain or Not to Pretrain: Examining the ...,"[Computer Science, Mathematics]",Pretraining NLP models with variants of Masked...,2020,ACL,18,"[Sinong Wang, Madian Khabsa, Hao Ma]","[Supervised learning, Language model, Document..."
2,True,10.18653/v1/W18-0211,Utilization of Nganasan digital resources: a s...,[Computer Science],"According to the wide-spread belief, although ...",2018,,8,[L. Fejes],"[Rounding, Lexicon, Tracer, Body of uterus, Cl..."
3,True,10.18653/v1/N19-1413,A large-scale study of the effects of word fre...,[Computer Science],A number of psycholinguistic studies have fact...,2019,NAACL,52,[Cory Shain],"[Word lists by frequency, Text corpus, Sentenc..."
4,False,10.18653/v1/2021.semeval-1.168,Amherst685 at SemEval-2021 Task 7: Joint Model...,[Computer Science],This paper describes our submission to theSemE...,2021,SEMEVAL,17,"[Brian Zylich, Akshay Gugnani, Gabriel Brookma...",[]


In [223]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9658 entries, 0 to 9657
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   doi              9658 non-null   object 
 1   title            9658 non-null   object 
 2   abstract         9499 non-null   object 
 3   authors          9658 non-null   object 
 4   venue            9658 non-null   object 
 5   year             9655 non-null   float64
 6   references       9658 non-null   int64  
 7   topics           9658 non-null   object 
 8   is_open_access   9658 non-null   bool   
 9   fields_of_study  9522 non-null   object 
 10  citations        9658 non-null   int64  
dtypes: bool(1), float64(1), int64(2), object(7)
memory usage: 764.1+ KB


(9658, 11)

In [224]:
#count missing values in columns
df.isna().sum()

doi                  0
title                0
abstract           159
authors              0
venue                0
year                 3
references           0
topics               0
is_open_access       0
fields_of_study    136
citations            0
dtype: int64

In [225]:
def fill_nan(df):
  df['abstract'] = df['abstract'].fillna("")
  df['year'] = df['year'].fillna(0)
  df['fields_of_study'] = df['fields_of_study'].apply(lambda d: d if isinstance(d, list) else [])
  return df

In [226]:
#Fill missing values with 0:
df = fill_nan(df)
df_test = fill_nan(df_test)

In [227]:
df[["abstract", "fields_of_study", "title"]]= df[["abstract", "fields_of_study", "title"]].astype(str)
df_test[["abstract", "fields_of_study", "title"]]= df[["abstract", "fields_of_study", "title"]].astype(str)

In [228]:
#Encode Author:
df["Author_count"] = df["authors"].apply(len).astype(float)
df_test["Author_count"] = df_test["authors"].apply(len).astype(float)

In [229]:
#Encode Year:
df["year_code"] = df["year"].apply(lambda x: 0.0 if x == 0.0 else x-1979.0 +1)
df_test["year_code"] = df_test["year"].apply(lambda x: 0.0 if x == 0.0 else x-1979.0 +1)

In [230]:
#Encode Title:
df["Title_code"] = df["title"].apply(len).astype(float)
df_test["Title_code"] = df_test["title"].apply(len).astype(float)

In [231]:
#Encode Topics:
df["Topics_code"] = df["topics"].apply(len).astype(float)
df_test["Topics_code"] = df_test["topics"].apply(len).astype(float)

In [232]:
df["is_open_access"]=df["is_open_access"].astype(float)
df_test["is_open_access"]=df_test["is_open_access"].astype(float)

In [233]:
df["references"]=df["references"].astype(float)
df_test["references"]=df_test["references"].astype(float)

In [234]:
df["citations"]=df["citations"].astype(float)

In [235]:
#Split X - Y:
X = df
Y = df.pop("citations")

In [236]:
#Encode Fields Of Study & Venue - Train:
from feature_engine.encoding import CountFrequencyEncoder
encoder1 = CountFrequencyEncoder(encoding_method='frequency', variables=["fields_of_study", "venue"])
encoder1.fit(X)

CountFrequencyEncoder(encoding_method='frequency',
                      variables=['fields_of_study', 'venue'])

In [237]:
X= encoder1.transform(X)

In [238]:
#Encode Fields Of Study & Venue - Test:
from feature_engine.encoding import CountFrequencyEncoder
encoder2 = CountFrequencyEncoder(encoding_method='frequency', variables=["fields_of_study", "venue"])
encoder2.fit(df_test)
df_test= encoder2.transform(df_test)

In [239]:
#Encode Abstract:
from category_encoders import MEstimateEncoder
encoder = MEstimateEncoder(cols = ["abstract"], m = 5.0)
encoder.fit(X, Y)
X= encoder.transform(X)
df_test= encoder.transform(df_test)

In [240]:
#Split DOI:
DOI = df_test.pop("doi")

In [241]:
X= X.drop(["doi", "authors", "year","is_open_access", "title", "topics"], axis = 1)
df_test = df_test.drop(["authors", "year","is_open_access", "title", "topics"], axis = 1)

In [242]:
print(X.shape)
print(df_test.shape)

(9658, 8)
(1000, 8)


In [243]:
X.head()

,abstract,venue,references,fields_of_study,Author_count,year_code,Title_code,Topics_code
0,40.829019,0.201595,39.0,0.928039,6.0,37.0,81.0,1.0
1,30.995686,0.000828,44.0,0.928039,5.0,42.0,124.0,0.0
2,31.662352,0.012011,30.0,0.928039,3.0,39.0,60.0,5.0
3,31.662352,0.007041,11.0,0.928039,2.0,39.0,87.0,6.0
4,32.495686,0.003106,26.0,0.928039,2.0,37.0,39.0,23.0


In [244]:
df_test = df_test[['abstract', 'venue', 'references', 'fields_of_study','Author_count',
       'year_code', 'Title_code', 'Topics_code']]

In [245]:
df_test.head()

,abstract,venue,references,fields_of_study,Author_count,year_code,Title_code,Topics_code
0,40.829019,0.037,47.0,0.925,3.0,43.0,81.0,11.0
1,30.995686,0.187,18.0,0.925,3.0,42.0,124.0,7.0
2,31.662352,0.056,8.0,0.925,1.0,40.0,60.0,6.0
3,31.662352,0.072,52.0,0.925,1.0,41.0,87.0,3.0
4,32.495686,0.010,17.0,0.925,4.0,43.0,39.0,0.0


In [246]:
#Standardizatin:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
scaler.fit(df_test)
X = scaler.transform(X)
df_test= scaler.transform(df_test)

X = pd.DataFrame(X)
df_test = pd.DataFrame(df_test)

In [247]:
#Evaluation code:
def score(Y_true, Y_pred):
    y_true = np.log1p(np.maximum(0, Y_true))
    y_pred = np.log1p(np.maximum(0, Y_pred))
    return 1 - np.mean((y_true-y_pred)**2) / np.mean((y_true-np.mean(y_true))**2)

#def evaluate(gold_path, pred_path):
    #gold = { x['doi']: x['citations'] for x in json.load(open(gold_path)) }
    #pred = { x['doi']: x['citations'] for x in json.load(open(pred_path)) }
    #y_true = np.array([ gold[key] for key in gold ])
    #y_pred = np.array([ pred[key] for key in gold ])
    #return score(y_true, y_pred)

In [181]:
a = np.array([1,2,3])
b = np.log1p(a)
c = np.expm1(b)
print(b)
print(c)

[0.69314718 1.09861229 1.38629436]
[1. 2. 3.]


In [182]:
a = np.array([1,2,3])
b = np.expm1(a)
c = np.log1p(b)
print(b)
print(c)

[ 1.71828183  6.3890561  19.08553692]
[1. 2. 3.]


In [183]:
# # Log-transform Y:
# Y = np.log1p(Y)
# Y

0       4.110874
1       0.693147
2       1.791759
3       1.791759
4       2.397895
          ...   
9653    2.197225
9654    0.693147
9655    0.693147
9656    1.386294
9657    5.433722
Name: citations, Length: 9658, dtype: float64

In [263]:
#Split data:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, train_size=0.8,test_size=0.2, random_state=666)

In [264]:
print(X_train.shape)
print(X_valid.shape)

(7726, 8)
(1932, 8)


In [265]:
y_valid

2007    289.0
615       0.0
9371     31.0
2497      0.0
4389     15.0
        ...  
1229      0.0
8322     11.0
9043      5.0
9070     11.0
8701      2.0
Name: citations, Length: 1932, dtype: float64

Model Application

In [266]:
# Linear Regression:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
LR = LinearRegression()
LR.fit(X_train, y_train)
LR_pred = LR.predict(X_valid)
LR_pred

array([286.83763676,   4.31028729,  38.20989094, ...,   8.81533546,
        10.29724318,   9.32724006])

In [267]:
print(r2_score(y_train, LR.predict(X_train)))
print(r2_score(y_valid, LR.predict(X_valid)))
print(score(y_valid, LR_pred))

0.9916869277694118
0.9795815264649435
0.7123035491429095


In [268]:
LR_pred = LR.predict(X_valid)
reversed_log = np.expm1(LR_pred)
reversed_log.shape

C:\Users\TL\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in expm1
  


(1932,)

In [84]:
reversed_log = reversed_log.reshape(1932,)

In [269]:
reversed_log

array([3.73252603e+124, 7.34618780e+001, 3.92956781e+016, ...,
       6.73576728e+003, 2.96497642e+004, 1.12390668e+004])

In [69]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()
parameters = {"n_neighbors" :[5, 10], 
              "weights": ["uniform", "distance"],
              "algorithm": ["auto", "kd_tree"],
               "p":[1,2,3]}
grid_knn = GridSearchCV(estimator = knn, param_grid = parameters, cv = 10, verbose = 1, n_jobs = -1)
Grid = grid_knn.fit(X_train, y_train)
Grid.best_params_

Fitting 10 folds for each of 24 candidates, totalling 240 fits


{'algorithm': 'auto', 'n_neighbors': 5, 'p': 3, 'weights': 'distance'}

In [71]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors = 5, weights = "distance", algorithm='auto', p=3)
knn.fit(X_train,y_train)
#knn.predict(X_valid)
print(r2_score(y_train, knn.predict(X_train)))
print(r2_score(y_valid, knn.predict(X_valid)))


1.0
0.9777543353828566


Running on test set:

In [275]:
# #Lasso:
# from sklearn import linear_model
# lasso = linear_model.Lasso(alpha=0.1, random_state=666)
# lasso.fit(X, Y)
# lasso_pred = lasso.predict(df_test)
# lasso_pred

In [42]:
#Linear:
from sklearn.linear_model import LinearRegression
LR = LinearRegression()
LR.fit(X,Y)
LR_pred_test = LR.predict(df_test)
LR_pred_test

array([[ 2.59587371],
       [ 1.39549787],
       [ 1.29078705],
       [ 1.8877122 ],
       [ 1.17864393],
       [ 1.50980432],
       [ 3.11550153],
       [ 0.62488458],
       [ 2.26079429],
       [ 1.87088515],
       [ 3.32761629],
       [ 2.59735778],
       [ 1.62321776],
       [ 1.77551953],
       [ 2.63131774],
       [ 1.82048326],
       [ 1.25988081],
       [ 1.30705012],
       [ 2.67336895],
       [ 1.635715  ],
       [ 1.94169395],
       [ 2.30393838],
       [ 3.72567643],
       [ 1.83343652],
       [ 2.19063402],
       [ 4.71549323],
       [ 3.37943391],
       [ 1.29996296],
       [ 1.17791438],
       [ 3.50280104],
       [ 2.13215655],
       [ 1.80362671],
       [ 2.57929795],
       [ 1.71475011],
       [ 6.69414328],
       [ 2.71108691],
       [ 2.33724645],
       [ 2.20840622],
       [ 1.94807495],
       [ 4.37748144],
       [ 2.4278188 ],
       [ 2.12004227],
       [ 2.29297439],
       [ 2.32212892],
       [ 1.48429614],
       [ 1

In [43]:
LR_pred_log = np.expm1(LR_pred_test)

In [44]:
LR_pred_log= LR_pred_log.reshape(1000,)   #Convert this into dict then json

In [110]:
# DOI = np.array(DOI)
# pred_test_linear= []
# for i, j in zip(DOI,LR_pred):
#     pred_test_linear.append({"doi": i, "citations":int(round(j, 0))})
# pred_test_linear

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()
parameters = {"n_neighbors" :[5, 10], 
              "weights": ["uniform", "distance"],
              "algorithm": ["auto", "kd_tree"],
               "p":[1,2,3]}
grid_knn = GridSearchCV(estimator = knn, param_grid = parameters, cv = 10, verbose = 1, n_jobs = -1)
Grid = grid_knn.fit(X, Y)
Grid.best_params_

Fitting 10 folds for each of 24 candidates, totalling 240 fits


{'algorithm': 'auto', 'n_neighbors': 10, 'p': 1, 'weights': 'distance'}

In [46]:
#KNN:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors = 10, weights = "distance", algorithm='auto', p=1)
knn.fit(X,Y)
knn_pred_test = knn.predict(df_test)
knn_pred_test

array([[3.10931897],
       [1.47808455],
       [1.77971806],
       [1.449788  ],
       [1.22707725],
       [0.60991752],
       [3.8460583 ],
       [2.35324978],
       [1.96125174],
       [1.0931308 ],
       [1.88052026],
       [3.71558977],
       [1.81425397],
       [2.04733265],
       [3.73342561],
       [2.09180791],
       [1.23590607],
       [1.23065167],
       [3.30634227],
       [1.71626945],
       [2.56417574],
       [1.44132837],
       [3.51170845],
       [1.24270214],
       [2.48684707],
       [4.43808304],
       [4.56441757],
       [0.75885207],
       [0.59164893],
       [3.17809714],
       [2.82737509],
       [1.64566769],
       [2.83591206],
       [1.03714514],
       [4.83282043],
       [2.39196389],
       [1.88232912],
       [2.51348466],
       [1.82895652],
       [5.10208285],
       [1.95476565],
       [1.57773946],
       [1.90333207],
       [2.49431869],
       [0.9946169 ],
       [0.64770896],
       [1.65399652],
       [2.608

In [47]:
knn_pred_log = np.expm1(knn_pred_test)

In [48]:
knn_pred_log = knn_pred_log.reshape(1000,)
knn_pred_log

array([2.14057802e+01, 3.38453925e+00, 4.92818477e+00, 3.26221085e+00,
       2.41124473e+00, 8.40279610e-01, 4.58081953e+01, 9.51970099e+00,
       6.10821915e+00, 1.98360053e+00, 5.55691525e+00, 4.00828095e+01,
       5.13649647e+00, 6.74720903e+00, 4.08221293e+01, 7.09954520e+00,
       2.44149535e+00, 2.42345979e+00, 2.62851411e+01, 4.56373394e+00,
       1.19899469e+01, 3.22630619e+00, 3.25054613e+01, 2.46496363e+00,
       1.10233076e+01, 8.36125869e+01, 9.50066605e+01, 1.13582304e+00,
       8.06965514e-01, 2.30010394e+01, 1.59010389e+01, 4.18447039e+00,
       1.60459401e+01, 1.82115153e+00, 1.24564607e+02, 9.93494790e+00,
       5.56878657e+00, 1.13478833e+01, 5.22738508e+00, 1.63363897e+02,
       6.06226381e+00, 3.84399336e+00, 5.70820946e+00, 1.11134776e+01,
       1.70368837e+00, 9.11157269e-01, 4.22783126e+00, 1.25729791e+01,
       1.58128568e+02, 2.06650999e+01, 2.11774238e+00, 1.46896366e+00,
       6.33813069e+01, 5.80396655e+01, 6.44209244e+00, 6.75171857e+01,
      

In [51]:
DOI = np.array(DOI)
pred_test = []
for i, j in zip(DOI,knn_pred_log):
    pred_test.append({"doi": i, "citations":int(j)})
pred_test

[{'doi': '10.18653/v1/2021.findings-acl.255', 'citations': 21},
 {'doi': '10.18653/v1/2020.acl-main.200', 'citations': 3},
 {'doi': '10.18653/v1/W18-0211', 'citations': 4},
 {'doi': '10.18653/v1/N19-1413', 'citations': 3},
 {'doi': '10.18653/v1/2021.semeval-1.168', 'citations': 2},
 {'doi': '10.18653/v1/2021.case-1.18', 'citations': 0},
 {'doi': '10.3115/1075218.1075228', 'citations': 45},
 {'doi': '10.18653/v1/P19-1572', 'citations': 9},
 {'doi': '10.18653/v1/W15-4208', 'citations': 6},
 {'doi': '10.18653/V1/2021.NAACL-MAIN.240', 'citations': 1},
 {'doi': '10.18653/v1/D15-1207', 'citations': 5},
 {'doi': '10.3115/1119303.1119310', 'citations': 40},
 {'doi': '10.18653/v1/2020.acl-main.758', 'citations': 5},
 {'doi': '10.18653/v1/2020.conll-1.15', 'citations': 6},
 {'doi': '10.3115/v1/N15-1163', 'citations': 40},
 {'doi': '10.18653/v1/2020.emnlp-main.279', 'citations': 7},
 {'doi': '10.18653/v1/W19-5057', 'citations': 2},
 {'doi': '10.18653/v1/W15-4311', 'citations': 2},
 {'doi': '10.18

In [115]:
import json
import numpy
import pandas
with open('predicted.json', 'w') as fout:
    json.dump(pred_test , fout)


In [116]:
f=open('predicted.json')
data=json.load(f)
for i in data:
    print(i)
f.close()

{'doi': '10.18653/v1/2021.findings-acl.255', 'citations': 21}
{'doi': '10.18653/v1/2020.acl-main.200', 'citations': 3}
{'doi': '10.18653/v1/W18-0211', 'citations': 5}
{'doi': '10.18653/v1/N19-1413', 'citations': 3}
{'doi': '10.18653/v1/2021.semeval-1.168', 'citations': 2}
{'doi': '10.18653/v1/2021.case-1.18', 'citations': 1}
{'doi': '10.3115/1075218.1075228', 'citations': 46}
{'doi': '10.18653/v1/P19-1572', 'citations': 10}
{'doi': '10.18653/v1/W15-4208', 'citations': 6}
{'doi': '10.18653/V1/2021.NAACL-MAIN.240', 'citations': 2}
{'doi': '10.18653/v1/D15-1207', 'citations': 6}
{'doi': '10.3115/1119303.1119310', 'citations': 40}
{'doi': '10.18653/v1/2020.acl-main.758', 'citations': 5}
{'doi': '10.18653/v1/2020.conll-1.15', 'citations': 7}
{'doi': '10.3115/v1/N15-1163', 'citations': 41}
{'doi': '10.18653/v1/2020.emnlp-main.279', 'citations': 7}
{'doi': '10.18653/v1/W19-5057', 'citations': 2}
{'doi': '10.18653/v1/W15-4311', 'citations': 2}
{'doi': '10.18653/V1/2020.COLING-MAIN.470', 'citat

In [ ]:
# from sklearn.preprocessing import FunctionTransformer
# transformer = FunctionTransformer(np.log1p)
# y_train_log = transformer.transform(y_train)
# y_valid_log = transformer.transform(y_valid)
# y_train_array = np.array(y_train_log).reshape(7726,)
# y_valid_array = np.array(y_valid_log).reshape(1932,)
# y_train_df = pd.DataFrame(y_train_array, columns = ["citations"])
# y_valid_df = pd.DataFrame(y_valid_array, columns = ["citations"])